In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/VRD-IU

/content/drive/MyDrive/VRD-IU


In [3]:
!cp train_visual_features.pkl train_textual_features.pkl  obj_cat.csv /content/

In [4]:
%cd /content/

/content


In [5]:
import pickle
with open("train_visual_features.pkl", 'rb') as f:
    visual_features = pickle.load(f)
with open("train_textual_features.pkl", 'rb') as f:
    textual_features = pickle.load(f)

In [6]:
from torch.utils.data import Dataset, DataLoader
import os
import torch
import numpy as np
import pandas as pd
class FeatureDataset(Dataset):
    def __init__(self, df,visual_features, textual_features):
        super().__init__()
        self.df = df
        self.visual_features = torch.tensor(list(visual_features[visual_features['id'].isin(train_df['id'])]['features']))
        self.textual_features = torch.tensor(list(textual_features[textual_features['id'].isin(train_df['id'])]['features']))

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        data = self.df.loc[index]
        visual_feats = self.visual_features[index]
        textual_feats = self.textual_features[index]
        return visual_feats, textual_feats, int(data['class'])


In [7]:
df = pd.read_csv("obj_cat.csv")
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df,test_size=0.2)

In [8]:
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [9]:
train_dataset = FeatureDataset(train_df,visual_features, textual_features)

In [10]:
val_dataset = FeatureDataset(val_df,visual_features, textual_features)

In [11]:
train_dataloader = DataLoader(train_dataset,batch_size=256,shuffle=True,num_workers=2)

In [12]:
val_dataloader = DataLoader(val_dataset,batch_size=256,shuffle=True,num_workers=2)

In [13]:
import torch.nn as nn
class Classifer(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fuser =  nn.Sequential(nn.Linear(1536+768,3072),
                                   nn.ReLU(),
                                   nn.Linear(3072,768))
        self.fc = nn.Linear(768,25)

    def forward(self, visual_feats, text_feats):
        embed = self.fuser(torch.cat((visual_feats,text_feats),dim=1))
        pred = self.fc(embed)
        return pred

In [14]:
from torch import optim
model = Classifer()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=1e-3)

In [19]:
def train(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0.0
    for visual_feats, text_feats, labels in dataloader:
        visual_feats = visual_feats.to(device)
        text_feats = text_feats.to(device)
        labels = labels.to(device)
        pred = model(visual_feats,text_feats)
        loss = criterion(pred,labels)
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return total_loss/len(dataloader)


In [20]:
from sklearn.metrics import f1_score, accuracy_score
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0.0
    total_acc = 0.0
    total_f1 = 0.0
    with torch.no_grad():
        for visual_feats, text_feats, labels in dataloader:
            visual_feats = visual_feats.to(device)
            text_feats = text_feats.to(device)
            labels = labels.to(device)
            pred = model(visual_feats,text_feats)
            loss = criterion(pred,labels)
            total_loss += loss.item()
            predicted_labels = pred.argmax(dim=1)
            f1 = f1_score(labels.detach().cpu().numpy(), predicted_labels.detach().cpu().numpy(),average="macro")
            acc = accuracy_score(labels.detach().cpu().numpy(), predicted_labels.detach().cpu().numpy())
            total_f1+=f1
            total_acc+=acc

    return total_loss/len(dataloader),total_acc/len(dataloader),total_f1/len(dataloader)

In [21]:
epochs = 100
from tqdm import tqdm
for epoch in tqdm(range(epochs)):
    train_loss = train(model,train_dataloader,criterion,optimizer)
    val_loss, acc, f1 = evaluate(model,val_dataloader,criterion)
    print(f'Epoch: {epoch} Train Loss: {train_loss} Val Loss: {val_loss} Val Accuracy: {acc} Val F1: {f1}')

  1%|          | 1/100 [00:03<05:23,  3.27s/it]

Epoch: 0 Train Loss: 2.1443084013812683 Val Loss: 2.137494858573465 Val Accuracy: 0.4676598714082407 Val F1: 0.030455218442424034


  2%|▏         | 2/100 [00:06<05:10,  3.16s/it]

Epoch: 1 Train Loss: 2.1357286511098637 Val Loss: 2.1322663461460785 Val Accuracy: 0.46840215420845754 Val F1: 0.030253217368888797


  3%|▎         | 3/100 [00:09<05:04,  3.14s/it]

...

Epoch: 17 Train Loss: 2.0847566145307876 Val Loss: 2.2237322470721077 Val Accuracy: 0.4637912120493359 Val F1: 0.03156484332587625


 19%|█▉        | 19/100 [01:07<04:40,  3.47s/it]

Epoch: 18 Train Loss: 2.0808235363048664 Val Loss: 2.220393917139839 Val Accuracy: 0.46390345452697207 Val F1: 0.03149896808261651


 19%|█▉        | 19/100 [01:09<04:55,  3.65s/it]


KeyboardInterrupt: 